In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


## Importing required libraries

In [2]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score

# Approach
1. Load data
2. Select useful features
3. Extract numerical and categorical columns
4. Make pipeline
5. Do Hyperparameter tuning with cross-validation
6. Check MAE
7. Find best parameters
8. Train model
9. Predicting `survived` on `test_data`
10. Submit predictions

## 1. Loading data

In [3]:
titanic_data = pd.read_csv("../input/titanic/train.csv", index_col="PassengerId")
test_data = pd.read_csv("../input/titanic/test.csv", index_col="PassengerId")

titanic_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 2. Selecting useful features

In [4]:
outliers = titanic_data[(titanic_data["Survived"]==1) & (titanic_data["Fare"] > 200)].index
titanic_data.drop(labels=outliers, inplace=True)

In [5]:
useful_features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Embarked"]
X = titanic_data[useful_features]
Y = titanic_data["Survived"]
test_data = test_data[useful_features]

## 3. Extracting numerical and categorical columns

In [6]:
num_cols = X.select_dtypes(exclude="object").columns
cat_cols = X.select_dtypes("object").columns
num_cols, cat_cols

(Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object'),
 Index(['Sex', 'Ticket', 'Embarked'], dtype='object'))

## 4. Making pipeline

In [7]:
class CreatePipeline:
    """Create Pipeline
    methods:
        pipeline: Create Final Pipeline
        
        create_model: Create the provided model
        
        numerical_transformer: Transform numerical cols
        
        categorical_transformer: Transform categorical cols \
        OneHotEncoding / OrdinalEncoding
        
        data_preprocessor: Preprocess the data using ColumnTransformer     
        """
    
    def pipeline(self, *, preprocessor, model, verbose=False):
        """Creates pipeline
        params:
            preprocessor
            model
        """
        steps = [("preprocessor", preprocessor),
                 ("model", model)]
        return Pipeline(steps=steps, verbose=verbose)
    
    
    def numerical_transformer(self, *, strategy="mean", **params):
        """Transform numerical columns using `SimpleImputer`.
        params:
            strategy: "mean" | "median" | "most_frequent" | "constant"
            **params: extra keyword args for SimpleImputer"""
        
        transformer = SimpleImputer(strategy=strategy, **params)
        return transformer

    
    def categorical_transformer(self, *, 
                                imp_strategy="most_frequent", 
                                encoder_type="Ordinal", 
                                imp_params={}, encoder_params={}):
        """Transform categorical columns by making Pipeline
        `SimpleImputer` | `OneHotEncoder` | `OrdinalEncoder`.
        args:
            imp_strategy: strategy for imputer values can be
                "most_frequent" | "constant"
            encoder_type: encoder type,
                "Ordinal" | "OneHot"
        kwargs:
            imp_params: keyword args for `SimpleImputer`.
            encoder_params: keyword args for encoder.`
        """
        if not encoder_type in ("Ordinal", "OneHot"):
            raise ValueError(f"Inappropriate value for encoder_type passed: {encoder_type}\
            Takes one of 'Ordinal' | 'OneHot'.")
        
        encoder = OrdinalEncoder if encoder_type=="Ordinal" else OneHotEncoder
        transformer = Pipeline(steps=[
            ("imputer", SimpleImputer(strategy=imp_strategy, **imp_params)),
            (encoder_type, encoder(**encoder_params))
        ])
        return transformer
    
    
    def data_preprocessor(self, *, transformers):
        """Preprocess the data using `ColumnTransformer`.
        Pass extact list of transformers
        to be passed in `ColumnTransformer`.
        each tuple consist of: (transformer_name,
                                transformer,
                                list_of_columns)."""
        preprocessor = ColumnTransformer(transformers=transformers)
        return preprocessor
    
    
    def create_model(self, *, model, random_state=0, n_estimators=1000, **kwargs):
        """Creates the model.
        **kwargs: keyword args for model."""
        my_model = model(random_state=random_state, n_estimators=n_estimators, **kwargs)
        return my_model

In [8]:
cp = CreatePipeline()
num_transformer = cp.numerical_transformer(strategy="median")
cat_transformer = cp.categorical_transformer(encoder_type="OneHot", encoder_params={"handle_unknown": "ignore"})
preprocessor = cp.data_preprocessor(
                    transformers=[("num", num_transformer, num_cols),
                                  ("cat", cat_transformer, cat_cols)
                                 ])

## 5. Doing hyperparameter tuning with Cross-validation using `XGBClassifier`

In [9]:
n_estimators = [350, 500, 750]
max_depths = [5, 10, 20]
learning_rate = [0.05, 0.1]
maes = {}
i = 0
for n in n_estimators:
    for md in max_depths:
        for rate in learning_rate:
            i += 1
            model = cp.create_model(model=XGBClassifier, n_estimators=n, max_depth=md, learning_rate=rate)
            pipeline = cp.pipeline(preprocessor=preprocessor, model=model)
            scores = -1 * cross_val_score(pipeline, X, Y, cv=10, verbose=True,
                                    scoring="neg_mean_absolute_error")
            mae = scores.mean()
            maes[i] = [n, md, rate, mae]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   15.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   14.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   17.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   16.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   20.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   18.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

## 7. Checking MAE

In [10]:
for i in maes:
    n, md, rate, mae = maes[i]
    print(f"{i}.\tN_estimators: {n}\tmax_depth: {md}\tlearning_rate: {rate}\tMAE: {mae}")

1.	N_estimators: 350	max_depth: 5	learning_rate: 0.05	MAE: 0.17108150470219435
2.	N_estimators: 350	max_depth: 5	learning_rate: 0.1	MAE: 0.1812957157784744
3.	N_estimators: 350	max_depth: 10	learning_rate: 0.05	MAE: 0.17900992685475442
4.	N_estimators: 350	max_depth: 10	learning_rate: 0.1	MAE: 0.19157523510971788
5.	N_estimators: 350	max_depth: 20	learning_rate: 0.05	MAE: 0.1949712643678161
6.	N_estimators: 350	max_depth: 20	learning_rate: 0.1	MAE: 0.19610762800417972
7.	N_estimators: 500	max_depth: 5	learning_rate: 0.05	MAE: 0.17450365726227793
8.	N_estimators: 500	max_depth: 5	learning_rate: 0.1	MAE: 0.1744644723092999
9.	N_estimators: 500	max_depth: 10	learning_rate: 0.05	MAE: 0.18585423197492162
10.	N_estimators: 500	max_depth: 10	learning_rate: 0.1	MAE: 0.19157523510971788
11.	N_estimators: 500	max_depth: 20	learning_rate: 0.05	MAE: 0.19041274817136886
12.	N_estimators: 500	max_depth: 20	learning_rate: 0.1	MAE: 0.19610762800417975
13.	N_estimators: 750	max_depth: 5	learning_rate: 

## 8. Best parameters
#### `n_estimators: 350`
#### `max_depth: 5`
#### `learning_rate: 0.05`
#### `MAE: 0.17108150470219435`

In [11]:
best_n_estimators = 350
best_max_depth = 5
best_rate = 0.05

## 9. Training `XGBClassifier` model with best parameters

In [12]:
model = cp.create_model(model=XGBClassifier, n_estimators=best_n_estimators, learning_rate=best_rate, max_depth=best_max_depth)
pipeline = cp.pipeline(preprocessor=preprocessor, model=model)
pipeline.fit(X, Y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='median'),
                                                  Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('OneHot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Sex', 'Ticket', 'Embarked'], dtype='object'))])),
                ('...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_const

## 10. Predicting `Survived` on `test_data`

In [13]:
test_preds = pipeline.predict(test_data)
test_preds

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

## 11. Submitting predictions

In [14]:
output = pd.DataFrame({"PassengerId": test_data.index, "Survived": test_preds})
output.to_csv("./submission_3.csv", index=False)